## Try different models and compare them
## 1. openai/whisper-base

In [ ]:
!pip install --upgrade pip
!pip install --upgrade transformers datasets[audio] accelerate


  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 76.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
ERROR: p

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

pipe = pipeline(
    "automatic-speech-recognition", model="openai/whisper-base", device=device
)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.81k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "hf-internal-testing/librispeech_asr_dummy", "clean", split="validation"
)
dataset


README.md:   0%|          | 0.00/520 [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/9.19M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/73 [00:00<?, ? examples/s]

Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 73
})

In [ ]:
from IPython.display import Audio

sample = dataset[2]

print(sample["text"])
Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

HE TELLS US THAT AT THIS FESTIVE SEASON OF THE YEAR WITH CHRISTMAS AND ROAST BEEF LOOMING BEFORE US SIMILES DRAWN FROM EATING AND ITS RESULTS OCCUR MOST READILY TO THE MIND


In [ ]:
sample_2 = dataset[4]

print(sample_2["text"])
Audio(sample_2["audio"]["array"], rate=sample_2["audio"]["sampling_rate"])

LINNELL'S PICTURES ARE A SORT OF UP GUARDS AND AT EM PAINTINGS AND MASON'S EXQUISITE IDYLLS ARE AS NATIONAL AS A JINGO POEM MISTER BIRKET FOSTER'S LANDSCAPES SMILE AT ONE MUCH IN THE SAME WAY THAT MISTER CARKER USED TO FLASH HIS TEETH AND MISTER JOHN COLLIER GIVES HIS SITTER A CHEERFUL SLAP ON THE BACK BEFORE HE SAYS LIKE A SHAMPOOER IN A TURKISH BATH NEXT MAN


In [ ]:
pipe(sample["audio"].copy(), max_new_tokens=256)

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


{'text': ' He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similarly is drawn from eating and its results occur most readily to the mind.'}

In [ ]:
pipe(sample_2["audio"].copy(), max_new_tokens=256)

{'text': " Linnell's pictures are a sort of upgards and atom paintings, and Mason's exquisite ittles are as national as a jingo poem. Mr. Birkett Foster's landscapes smile at one much in the same way that Mr. Carker used to flash his teeth. And Mr. John Collier gives his sitter a cheerful slap on the back before he says like a shampooer in a Turkish bath. Next man"}

#### Previous model makes small errors in transcription but is pretty fast

## 2. openai/whisper-small

In [ ]:
pipe_2 = pipeline("automatic-speech-recognition", model="openai/whisper-small",device=device)


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [ ]:
pipe_2(sample["audio"].copy(), max_new_tokens=256)

{'text': ' He tells us that at this festive season of the year, with Christmas and roast beef looming before us, symbolies drawn from eating and its results occur most readily to the mind.'}

In [ ]:
pipe_2(sample_2["audio"].copy(), max_new_tokens=256)

{'text': " Linnell's pictures are a sort of Up Guards and Adam paintings, and Mason's exquisite idles are as national as a jingle poem. Mr. Birkut Foster's landscapes smile at one much in the same way that Mr. Karker used to flash his teeth. And Mr. John Collier gives his sitter a cheerful slap on the back, before he says, like a shampoo or a Turkish bath, next man"}

#### second model still has errors with colloquial language and is slower than the first one

## 3. openai/whisper-medium

In [ ]:
pipe_3 = pipeline("automatic-speech-recognition", model="openai/whisper-medium",device=device)

config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [ ]:
pipe_3(sample["audio"].copy(), max_new_tokens=256)

{'text': ' He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similes drawn from eating and its results occur most readily to the mind.'}

In [ ]:
pipe_3(sample_2["audio"].copy(), max_new_tokens=256)

{'text': " Linnell's pictures are a sort of Up Guards and Adam paintings, and Mason's exquisite idles are as national as a jingo poem. Mr. Birkett Foster's landscapes smile at one much in the same way that Mr. Carcar used to flash his teeth. And Mr. John Collier gives his sitter a cheerful slap in the back before he says, like a shampooer in a Turkish bath, Next man!"}

#### third model seems to have less errors with colloquial language and has comparable speed to the second model

#### Evaluating the models using WER

In [ ]:
!pip install --upgrade evaluate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 72.0 MB/s eta 0:00:00


In [ ]:
samples = dataset

In [ ]:
samples["text"][0]

'MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL'

In [ ]:
predictions_1 = pipe(samples["audio"].copy(), max_new_tokens=412)
predictions_2 = pipe_2(samples["audio"].copy(), max_new_tokens=412)
predictions_3 = pipe_3(samples["audio"].copy(), max_new_tokens=412)

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


In [ ]:
def extract_text_from_predictions(predictions):
  return [prediction['text'] for prediction in predictions]


text_predictions_1 = extract_text_from_predictions(predictions_1)
text_predictions_2 = extract_text_from_predictions(predictions_2)
text_predictions_3 = extract_text_from_predictions(predictions_3)


In [ ]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()

normalized_predictions_1 = [normalizer(pred) for pred in text_predictions_1]
normalized_predictions_2 = [normalizer(pred) for pred in text_predictions_2]
normalized_predictions_3 = [normalizer(pred) for pred in text_predictions_3]
all_references_norm = [normalizer(label) for label in samples["text"]]


In [ ]:
all_references_norm[0]

'mister quilter is the apostle of the middle classes and we are glad to welcome his gospel'

In [ ]:
normalized_predictions_1[0]

' mr quilter is the apostle of the middle classes and we are glad to welcome his gospel '

In [ ]:
from evaluate import load

wer_metric = load("wer")

wer = 100 * wer_metric.compute(references = all_references_norm, predictions = normalized_predictions_1)

print(f"wer for first model is {wer}")

wer for first model is 9.49529512403764


In [ ]:
wer_2 = 100 * wer_metric.compute(references = all_references_norm, predictions = normalized_predictions_2)
print(f"wer for second model is {wer_2}")

wer for second model is 7.442258340461934


In [ ]:
wer_3 = 100 * wer_metric.compute(references = all_references_norm, predictions = normalized_predictions_3)
print(f"wer for third model is {wer_3}")

wer for third model is 5.9880239520958085


#### The third model is the most accurate while still having a good inference time

## Use the third model for speech-to-text in arabic

In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
from datasets import load_dataset

common_voice_test = load_dataset(
    "mozilla-foundation/common_voice_13_0", "ar", split="test"
)

common_voice_13_0.py:   0%|          | 0.00/8.18k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/14.7k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.65k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/65.4k [00:00<?, ?B/s]

The repository for mozilla-foundation/common_voice_13_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_13_0.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


...


Reading metadata...: 10445it [00:00, 114472.70it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 12432it [00:00, 124291.08it/s]
Reading metadata...: 36192it [00:00, 122714.37it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 14994it [00:00, 116165.83it/s]


In [ ]:
ar_samples = common_voice_test.select(range(10))

ar_sample = ar_samples[0]

ar_sample

{'client_id': '0237f666c0b4f0779056b7c9289e40c170c6b0974b4214fed33f00a1f2c61671365317fcfb9788033286584f211b20fdd8c3e5757c404b599b1578b57f752c4f',
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/4cafd49f4de165179477e1f6afd58f7cbd31f66a2ce6f14aa13294a3f252e931/ar_test_0/common_voice_ar_28865270.mp3',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/4cafd49f4de165179477e1f6afd58f7cbd31f66a2ce6f14aa13294a3f252e931/ar_test_0/common_voice_ar_28865270.mp3',
  'array': array([ 3.55271368e-15, -1.17239551e-13, -1.98951966e-13, ...,
          5.80284450e-06,  1.28727488e-05,  9.77996660e-06]),
  'sampling_rate': 48000},
 'sentence': 'زارني في أوائل الشهر بدري',
 'up_votes': 2,
 'down_votes': 0,
 'age': 'twenties',
 'gender': 'male',
 'accent': '',
 'locale': 'ar',
 'segment': '',
 'variant': ''}

In [ ]:
print(ar_sample["sentence"])
Audio(ar_sample["audio"]["path"], rate=ar_sample["audio"]["sampling_rate"])

زارني في أوائل الشهر بدري


In [ ]:
ar_output = pipe_3(ar_sample["audio"].copy(), max_new_tokens=256)

In [ ]:
print(ar_output)

{'text': ' زارني في أوائل الشهر بدري'}


In [ ]:
ar_sample = ar_samples[4]
print(ar_sample["sentence"])
Audio(ar_sample["audio"]["path"], rate=ar_sample["audio"]["sampling_rate"])


ألديك قلم ؟


In [ ]:
ar_output = pipe_3(ar_sample["audio"].copy(), max_new_tokens=256)
print(ar_output)

{'text': ' ألديك أخضم'}


In [ ]:
ar_sample = ar_samples[8]
print(ar_sample["sentence"])
Audio(ar_sample["audio"]["path"], rate=ar_sample["audio"]["sampling_rate"])


الآن قد استوعبتها


In [ ]:
ar_output = pipe_3(ar_sample["audio"].copy(), max_new_tokens=256)
print(ar_output)

{'text': ' الآن قد تتعبتها'}


In [ ]:
ar_sample = ar_samples[9]
print(ar_sample["sentence"])
Audio(ar_sample["audio"]["path"], rate=ar_sample["audio"]["sampling_rate"])


مات الملك!


In [ ]:
ar_output = pipe_3(ar_sample["audio"].copy(), max_new_tokens=256)
print(ar_output)

{'text': ' مات الملك'}
